In [109]:
import pandas as pd
import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# 数据读入和预处理

In [110]:
list_sheet = ['2015','2016','2017','2018','2019','2020','2021']
list_df = []
for i in range(len(list_sheet)):
    i_df = pd.read_excel('cma_tc_forecast_analysis_2015-2021.xlsx',sheet_name=list_sheet[i])
    list_df.append(i_df)
df  = pd.concat(list_df)
df.index =  range(len(df))

In [112]:
#构造新的时间特征
hour  = pd.to_datetime(df['dateUTC'], format='%Y%m%d%H%M').dt.hour
month = pd.to_datetime(df['dateUTC'], format='%Y%m%d%H%M').dt.month

df.loc[:,'hour']  = hour.values
df.loc[:,'month'] = month.values

In [115]:
X = df[['forecast_hour', 'lonTC_f', 'latTC_f', 'mslp_f', 'vmax_f','hour', 'month' ]]
Y = df[['lonTC_a', 'latTC_a', 'mslp_a', 'vmax_a',]]

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25) #分离训练集和测试集

#测试集的路径，中心气压。最大风速，用于结果对比
Y_fcst  = X_test[['lonTC_f', 'latTC_f', 'mslp_f', 'vmax_f']]
Y_anal  = Y_test
Y_fcst.index = range(len(Y_fcst))
Y_anal.index = range(len(Y_anal))

# 误差计算函数

In [117]:
#半正矢公式计算球面距离（用于计算台风路径误差）
def haversine_dist(lat1,lng1,lat2,lng2):
  lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
  radius = 6371  # Earth's radius taken from google
  lat = lat2 - lat1
  lng = lng2 - lng1
  d = np.sin(lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng/2) ** 2
  h = 2 * radius * np.arcsin(np.sqrt(d))
  return h

In [118]:
#传入Pandas dataframe数据，计算误差
def calc_rmse(Y_pred, Y_fcst, Y_anal):
    # pred : AI订正后的数据
    # fcst : CMA业务预报数据
    # anal : CMA业务实况分析
    
    #计算路径误差
    ERR_dist_pred = haversine_dist(Y_pred['latTC_p'], Y_pred['lonTC_p'], Y_anal['latTC_a'], Y_anal['lonTC_a'])
    ERR_dist_fcst = haversine_dist(Y_fcst['latTC_f'], Y_fcst['lonTC_f'], Y_anal['latTC_a'], Y_anal['lonTC_a'])
    MSE_dist_pred = (ERR_dist_pred.abs()**2).mean()
    MSE_dist_fcst = (ERR_dist_fcst.abs()**2).mean()
    RMSE_dist_pred = np.sqrt(MSE_dist_pred)
    RMSE_dist_fcst = np.sqrt(MSE_dist_fcst)
    improve_track  = ( RMSE_dist_fcst-RMSE_dist_pred)/ RMSE_dist_fcst*100 # %
   
    
    #计算中心气压误差
    MSE_mslp_pred  = mean_squared_error(Y_pred['mslp_p'], Y_anal['mslp_a'])
    MSE_mslp_fcst  = mean_squared_error(Y_fcst['mslp_f'], Y_anal['mslp_a'])
    RMSE_mslp_pred = np.sqrt(MSE_mslp_pred)
    RMSE_mslp_fcst = np.sqrt(MSE_mslp_fcst)
    improve_mslp   = ( RMSE_mslp_fcst-RMSE_mslp_pred)/ RMSE_mslp_fcst*100 # %
    
    #计算最大风速误差
    MSE_vmax_pred  = mean_squared_error(Y_pred['vmax_p'], Y_anal['vmax_a'])
    MSE_vmax_fcst  = mean_squared_error(Y_fcst['vmax_f'], Y_anal['vmax_a'])
    RMSE_vmax_pred = np.sqrt(MSE_vmax_pred)
    RMSE_vmax_fcst = np.sqrt(MSE_vmax_fcst)
    improve_vmax   = ( RMSE_vmax_fcst-RMSE_vmax_pred)/ RMSE_vmax_fcst*100 # %
    
    print("!!!!!-------台风路径-------!!!!!")
    print("Track fcst RMSE: ", '%.2f'%RMSE_dist_fcst, 'km')
    print("Track pred RMSE: ", '%.2f'%RMSE_dist_pred, 'km')
    print("Track improve: ",'%.2f'%improve_track, '%' )
    print(" ")
    
    print("!!!!!-------中心气压-------!!!!!")
    print("MSLP fcst RMSE: ", '%.2f'%RMSE_mslp_fcst, 'hPa')
    print("MSLP pred RMSE: ", '%.2f'%RMSE_mslp_pred, 'hPa')
    print("MSLP improve: ",'%.2f'%improve_mslp, '%' )
    print(" ")
    
    print("!!!!!-------最大风速-------!!!!!")
    print("Vmax fcst RMSE: ", '%.2f'%RMSE_vmax_fcst, 'm/s')
    print("Vmax pred RMSE: ", '%.2f'%RMSE_vmax_pred, 'm/s')
    print("Vmax improve: ",'%.2f'%improve_vmax, '%' )
    print(" ")
    
    return None

# 随机森林订正
袋外样本oob (Out of bag)，检测模型的泛化能力，和交叉验证类似。可以理解成从train datasets 中分出来的validation datasets。

In [119]:
def RFModel_predict(X_train, Y_train):
    rfr = RandomForestRegressor(
          n_estimators = 100,
          random_state = 0,
          n_jobs = -1,
          oob_score = True
    )
    rfr = rfr.fit(X_train, Y_train)
    print('oob_score: ', rfr.oob_score_)
    importance = pd.DataFrame.from_dict(
      {name: value for (name, value) in zip(X_train.columns, rfr.feature_importances_)},
      orient='index', columns=['importance'])

    importance.sort_values('importance', ascending=False, inplace=True)
    print(importance)
    return rfr, importance


In [120]:
rfr, importance = RFModel_predict(X_train, Y_train)
Y_pred =  rfr.predict(X_test)

oob_score:  0.9683824558690758
               importance
vmax_f           0.529605
lonTC_f          0.220978
latTC_f          0.124504
mslp_f           0.068949
month            0.032334
forecast_hour    0.012149
hour             0.011481


In [121]:
# #numpy转为dataframe
Y_pred = pd.DataFrame(Y_pred,columns=['lonTC_p', 'latTC_p', 'mslp_p', 'vmax_p'])

# 误差计算

In [122]:
#计算误差
calc_rmse(Y_pred, Y_fcst, Y_anal)

!!!!!-------台风路径-------!!!!!
Track fcst RMSE:  135.79 km
Track pred RMSE:  83.59 km
Track improve:  38.44 %
 
!!!!!-------中心气压-------!!!!!
MSLP fcst RMSE:  11.02 hPa
MSLP pred RMSE:  5.40 hPa
MSLP improve:  51.00 %
 
!!!!!-------最大风速-------!!!!!
Vmax fcst RMSE:  5.86 m/s
Vmax pred RMSE:  2.92 m/s
Vmax improve:  50.16 %
 


# 2106号台风台风“烟花”的订正结果

In [153]:
df_INFA = df[df.tc_num == 2106]  #筛选2106号台风数据

###构造X输入
hour  = pd.to_datetime(df_INFA['dateUTC'], format='%Y%m%d%H%M').dt.hour
month = pd.to_datetime(df_INFA['dateUTC'], format='%Y%m%d%H%M').dt.month

df_INFA.loc[:,'hour']  = hour.values
df_INFA.loc[:,'month'] = month.values

X_INFA = df_INFA[['forecast_hour', 'lonTC_f', 'latTC_f', 'mslp_f', 'vmax_f','hour', 'month' ]]
###模型预测
Y_pred =  rfr.predict(X_INFA)  
# #numpy转为dataframe
Y_pred = pd.DataFrame(Y_pred,columns=['lonTC_p', 'latTC_p', 'mslp_p', 'vmax_p'])


In [154]:
#预报数据和实况数据，用于计算误差
Y_fcst  = df_INFA[['lonTC_f', 'latTC_f', 'mslp_f', 'vmax_f']]
Y_anal  = df_INFA[['lonTC_a', 'latTC_a', 'mslp_a', 'vmax_a',]]
Y_fcst.index = range(len(Y_fcst))
Y_anal.index = range(len(Y_anal))

In [212]:
#计算误差
calc_rmse(Y_pred, Y_fcst, Y_anal)

!!!!!-------台风路径-------!!!!!
Track fcst RMSE:  107.56 km
Track pred RMSE:  19.89 km
Track improve:  81.50 %
 
!!!!!-------中心气压-------!!!!!
MSLP fcst RMSE:  9.62 hPa
MSLP pred RMSE:  1.18 hPa
MSLP improve:  87.75 %
 
!!!!!-------最大风速-------!!!!!
Vmax fcst RMSE:  4.48 m/s
Vmax pred RMSE:  0.71 m/s
Vmax improve:  84.05 %
 
